In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163698 sha256=a6e40c0f60597e15019e6a62611c06c60acbfc923e4b13e71dbf297c1cd8c694
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


# Surprise를 이용한 추천 시스템 구축

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [3]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


- SVD로 잠재 요인 협업 필터링 수행

In [5]:
algo = SVD()
algo.fit(trainset)

- test 메서드

In [6]:
predictions = algo.test(testset)
print('prediction type: ', type(predictions), ' size: ', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type:  <class 'list'>  size:  25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.2671687604905455, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.738095090886196, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.339938991869213, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.7550105268957505, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.376413114820611, details={'was_impossible': False})]

In [7]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.2671687604905455),
 ('882', '291', 3.738095090886196),
 ('535', '507', 4.339938991869213)]

- predict 메서드

In [8]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 3.74   {'was_impossible': False}


- accuracy: 성능 평가 정보 제공

In [9]:
accuracy.rmse(predictions)

RMSE: 0.9473


0.947319380067354

# Surprise 주요 모듈 소개

- ratings_noh.csv : 인덱스, 헤더 모두 제거된 파일

In [30]:
import pandas as pd

ratings = pd.read_csv('/content/ratings.csv')
ratings.to_csv('/content/ratings_noh.csv', index=False, header=False)

- Reader 클래스를 이용해 데이터 파일의 파싱 포맷을 정의
  - 4개의 칼럼 명시
  - 분리 문자: 콤마
  - 평점 단위: 0.5, 최대 평점: 5
- load_from_file()로 생성된 Reader 객체 참조해 데이터 파일 파싱하면서 로딩

In [22]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('/content/ratings_noh.csv', reader=reader)

- SVD 행렬 분해 기법을 사용해 추천 예측
- RMSE로 예측 평점과 실제 평점 데이터 평가

In [23]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8702


0.870235810382944

# 판다스 DataFrame에서 Surprise 데이터 세트로 로딩

In [34]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

# 교차 검증과 하이퍼 파라미터 튜닝

- cross_validate() 이용해 교차 검증 수행
  - 인자로 알고리즘 객체, 데이터, 성능 평가 방법, 폴드 데이터 세트 개수 입력
- RMSE, MAE로 성능 평가

In [32]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('/content/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8812  0.8775  0.8653  0.8719  0.8663  0.8724  0.0062  
MAE (testset)     0.6763  0.6732  0.6670  0.6693  0.6655  0.6703  0.0040  
Fit time          1.50    1.56    1.54    1.89    1.76    1.65    0.15    
Test time         0.12    0.21    0.14    0.12    1.43    0.41    0.51    


{'test_rmse': array([0.88115471, 0.87746904, 0.86533978, 0.8718613 , 0.86628275]),
 'test_mae': array([0.6763067 , 0.67322151, 0.66695583, 0.66933729, 0.66551671]),
 'fit_time': (1.4981377124786377,
  1.5601286888122559,
  1.5423355102539062,
  1.885037899017334,
  1.7589476108551025),
 'test_time': (0.12156105041503906,
  0.21245408058166504,
  0.13970565795898438,
  0.1236717700958252,
  1.43387770652771)}

In [35]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [20,40,60], 'n_factors': [50,100,200]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8758129496654008
{'n_epochs': 20, 'n_factors': 50}


# Surprise를 이용한 개인화 영화 추천 시스템 구축

- 학습/테스트 데이터로 분리하지 않고 전체를 학습 데이터로 사용

In [36]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'n_users'

- Surprise는 train_test_split()을 이용해 내부에서 사용하는 TrainSet 클래스로 반환하지 않으면 fit()을 통해 학습할 수 없음

  => error
- 데이터 세트 전체를 학습 데이터로 사용하려면 DatasetAutoFolds 클래스 이용

In [37]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data_folds = DatasetAutoFolds(ratings_file='/content/ratings_noh.csv', reader=reader)

trainset = data_folds.build_full_trainset()

- SVD 이용해 학습 수행

In [38]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

- 특정 사용자: userId=9인 사용자
- 예측 수행 위해 userId 9가 아직 평점 매기지 않은 영화를 movieId 42로 선정

In [39]:
movies = pd.read_csv('/content/movies.csv')

movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [40]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


- 추천 예측 평점: 3.13

- 평점 매기지 않은 전체 영화 추출 후 예측 평점 순으로 영화 추천

In [41]:
def get_unseen_surprise(ratings, movies, userId):
  seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()

  total_movies = movies['movieId'].tolist()

  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수: ', len(seen_movies), '추천 대상 영화 수: ', len(unseen_movies), '전체영화 수: ', len(total_movies))

  return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수:  46 추천 대상 영화 수:  9696 전체영화 수:  9742


- 추천 대상 영화 모두를 대상으로 추천 알고리즘 객체의 predict() 메서드 호출, 그 결과인 Prediction 객체를 리스트 객체로 저장
- Prediction 객체를 예측 평점 높은 순으로 다시 정렬
- Top-N개의 Prediction 객체에서 영화 아이디, 영화 제목, 예측 평점 정보 추출해 반환

In [42]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
  def sortkey_est(pred):
    return pred.est

  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions = predictions[:top_n]

  top_movie_ids = [int(pred.iid) for pred in top_predictions]
  top_movie_rating = [pred.est for pred in top_predictions]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

  top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
  return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('#### Top-10 추천 영화 리스트 ####')
for top_movie in top_movie_preds:
  print(top_movie[1], ":", top_movie[2])

평점 매긴 영화 수:  46 추천 대상 영화 수:  9696 전체영화 수:  9742
#### Top-10 추천 영화 리스트 ####
Usual Suspects, The (1995) : 4.234330738920852
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) : 4.1968526561436885
Godfather, The (1972) : 4.133438141385809
Reservoir Dogs (1992) : 4.110849633234617
Streetcar Named Desire, A (1951) : 4.10483053201223
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.093330416871621
Lawrence of Arabia (1962) : 4.088512459573022
Apocalypse Now (1979) : 4.085797951503402
Goodfellas (1990) : 4.0763518422864
All the President's Men (1976) : 4.062344448333289
